In [1]:
import pandas as pd

import camelot

import pdfplumber

import os

In [2]:
def process_all_pdfs_in_folder(source_path, sink_path, desired_disease):
    year_df = []
    for filename in os.listdir(source_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(source_path, filename)
            print(f"Processando o arquivo: {pdf_path}")
            week_df = extract_and_process_tables(pdf_path, desired_disease, filename)
            year_df.append(week_df)
            
    if year_df:
        concatenated_df = pd.concat(year_df, ignore_index=True)

        pdf_filename = source_path.rsplit('/', 1)[-1]
        print("Salvando")
        csv_filename = os.path.splitext(pdf_filename)[0] + desired_disease + '.csv'
        output_path = os.path.join(sink_path, csv_filename)
        concatenated_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Arquivo salvo: {csv_filename}")
            

In [19]:
def extract_and_process_tables(pdf_path, desired_disease, filename):
    all_dfs = []
    pdf = pdfplumber.open(pdf_path)

    for page_num in range(len(pdf.pages)):
        try:
            tables = camelot.read_pdf(pdf_path, pages=str(page_num + 1), flavor='stream')

            if tables:
                df = tables[0].df
                page = pdf.pages[page_num]
                table = page.extract_table()
                df_header = pd.DataFrame(table)
                                
                if (not df_header.empty 
                    and df_header.shape[0] > 1  # Verifica se há pelo menos 2 linhas
                    and df_header.shape[1] > 1  # Verifica se há pelo menos 2 colunas
                    and df_header.iloc[0, 1] is not None 
                    and isinstance(df_header.iloc[0, 1], str)
                    and desired_disease in df_header.iloc[0, 1]):  
                    
                    first = None
                    

                    for n in range(4, 10):
                        if df.iloc[n, 0]:
                            first = n
                            break
                            
                    df_all = process_table(df, df_header, filename, first)
                    
                        
                    if not df_all.empty:
                        all_dfs.append(df_all)
#                         print(all_dfs)
                        print(f"Processando a página {page_num + 1} com a tabela desejada.")

        except Exception as e:
            print(f"Erro ao processar a página {page_num + 1}: {e}")

    pdf.close()

    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
    else:
        final_df = pd.DataFrame()

    # Salva o DataFrame final em um arquivo CSV
#     pdf_filename = os.path.basename(pdf_path)
#     csv_filename = os.path.splitext(pdf_filename)[0] + '_filtered.csv'
#     output_path = os.path.join('/home/pirata/Documents/projects/epidemic_database/bases', csv_filename)
#     final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    return final_df

In [22]:
# Defina a função para processar as tabelas do PDF
def process_table(df, df_header, filename, first):
    
    
    if df.empty or df_header.empty:
        print("DataFrame ou cabeçalho vazio.")
        return pd.DataFrame()  # Retorna um DataFrame vazio se não houver dados
    
    weeknumber = filename[3:5]
    

    df = df.iloc[first:].reset_index(drop=True)
    df.columns = df.iloc[0]
    # df = df[:]

    col_names = [f'Coluna{i+1}' for i in range(len(df.columns))]
    df.columns = col_names

    for col in col_names[1:10]:
        df[col] = df[col].str.replace(' ', '')

    first_col_name = df.columns[0]
    df = df[~df[first_col_name].str.match(r'^(TOTAL|FUENTE.*|§FUENTE.*)')]

    indices_para_converter = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    for idx in indices_para_converter:
        if idx < len(df.columns):  # Verifica se o índice está dentro dos limites
            df.iloc[:, idx] = pd.to_numeric(df.iloc[:, idx], errors='coerce').fillna(0).astype(int)

    num_col = len(df.columns)

    if num_col == 12:

        df = df.drop(['Coluna3','Coluna5'],axis = 1)

    df1 = df.iloc[:, 0:5].copy()
    if len(df_header) > 1 and len(df_header.columns) > 1:
        df1['disease'] = df_header.iloc[0, 1].replace('\n', ' ')
        df1['year'] = df_header.iloc[1, 1].replace('\n', ' ')
    else:
        df1['disease'] = ''
        df1['year'] = ''

    if num_col == 12:

        df1['MF'] = df1.get('Coluna4', 0) + df1.get('Coluna6', 0)

    else:

        df1['MF'] = df1.get('Coluna3', 0) + df1.get('Coluna4', 0)   


    df1['Acum_Year'] = int(df_header.iloc[1, 1].replace('\n', ' ')) - 1
    df1['week'] = weeknumber
    df1['En_Estudio'] = ''

    novos_nomes = [
        df_header.iloc[0, 0].replace('\n', ' ') if len(df_header) > 0 and len(df_header.columns) > 0 else '',
        df_header.iloc[3, 1].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 1 else '',
        df_header.iloc[4, 2].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 2 else '',
        df_header.iloc[4, 3].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 3 else '',
        'Acum.'
    ]

    df1.columns = novos_nomes + list(df1.columns[5:])
    column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9, 10]
    df1 = df1.iloc[:, column_order]

    df2 = df.iloc[:, [0] + list(range(5, 10))].copy()
    if len(df_header) > 0 and len(df_header.columns) > 5:
        df2['disease'] = df_header.iloc[0, 5].replace('\n', ' ')
        df2['year'] = df_header.iloc[1, 5].replace('\n', ' ')
    else:
        df2['disease'] = ''
        df2['year'] = ''

    if num_col == 12:

        df2['MF'] = df2.get('Coluna10', 0) + df2.get('Coluna11', 0)

    else:

        df2['MF'] = df2.get('Coluna8', 0) + df2.get('Coluna9', 0)


    df2['Acum_Year'] = int(df_header.iloc[1, 5].replace('\n', ' ')) - 1

    df2['week'] = weeknumber

    column_order = [0, 2, 3, 4, 8, 5, 9, 6, 7, 10, 1]

    df2 = df2.iloc[:, column_order]

    novos_nomes = [
        df_header.iloc[0, 0].replace('\n', ' ') if len(df_header) > 0 and len(df_header.columns) > 0 else '',
        df_header.iloc[3, 1].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 1 else '',
        df_header.iloc[4, 2].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 2 else '',
        df_header.iloc[4, 3].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 3 else '',
        'MF',
        'Acum.'
    ]

    df2.columns = novos_nomes + list(df2.columns[6:10]) + ['En_Estudio']
    
    
    df_all = pd.concat([df1, df2], axis=0, ignore_index=True)
    

    return df_all

In [24]:

source_path = '/home/pirata/Documents/projects/epidemic_database/files/2015'
sink_path = '/home/pirata/Documents/projects/epidemic_database/bases/Fiebre por Dengue'
desired_disease = 'Fiebre por Dengue'
process_all_pdfs_in_folder(source_path, sink_path, desired_disease)

Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem43.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem37.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem24.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem42.pdf


/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 2
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem51.pdf
Processando a página 27 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem04.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem35.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem49.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem14.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem08.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem03.pdf
Processando a

/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem23.pdf
Processando a página 27 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem02.pdf
Processando a página 30 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem22.pdf


...

Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem38.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem44.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem17.pdf


/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 3
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem30.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem16.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem21.pdf
Processando a página 27 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem36.pdf


/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 2
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem26.pdf
Processando a página 29 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem15.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem12.pdf
Processando a página 27 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem45.pdf
Processando a página 27 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem40.pdf
Processando a página 29 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem48.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2015/sem01.pdf
Processando a

In [13]:
filename = 'sem43.pdf'

pdf_path = '/home/pirata/Documents/projects/epidemic_database/files/2015_teste/sem43.pdf'

page_num = 30

pdf = pdfplumber.open(pdf_path)

tables = camelot.read_pdf(pdf_path, pages=str(page_num + 1), flavor='stream')

if tables:
    dfx = tables[0].df
    page = pdf.pages[page_num]
    table = page.extract_table()
    df_header = pd.DataFrame(table)

In [11]:
weeknumber = filename[3:5]

df = dfx.iloc[5:].reset_index(drop=True)
df.columns = df.iloc[0]
# df = df[:]

col_names = [f'Coluna{i+1}' for i in range(len(df.columns))]
df.columns = col_names

for col in col_names[1:10]:
    df[col] = df[col].str.replace(' ', '')

first_col_name = df.columns[0]
df = df[~df[first_col_name].str.match(r'^(TOTAL|FUENTE.*)')]

indices_para_converter = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
for idx in indices_para_converter:
    if idx < len(df.columns):  # Verifica se o índice está dentro dos limites
        df.iloc[:, idx] = pd.to_numeric(df.iloc[:, idx], errors='coerce').fillna(0).astype(int)
        
num_col = len(df.columns)
        
if num_col == 12:
    
    df = df.drop(['Coluna3','Coluna5'],axis = 1)

df1 = df.iloc[:, 0:5].copy()
if len(df_header) > 1 and len(df_header.columns) > 1:
    df1['disease'] = df_header.iloc[0, 1].replace('\n', ' ')
    df1['year'] = df_header.iloc[1, 1].replace('\n', ' ')
else:
    df1['disease'] = ''
    df1['year'] = ''
    
if num_col == 12:
    
    df1['MF'] = df1.get('Coluna4', 0) + df1.get('Coluna6', 0)
    
else:
    
    df1['MF'] = df1.get('Coluna3', 0) + df1.get('Coluna4', 0)   


df1['Acum_Year'] = int(df_header.iloc[1, 1].replace('\n', ' ')) - 1
df1['week'] = weeknumber
df1['En_Estudio'] = ''

novos_nomes = [
    df_header.iloc[0, 0].replace('\n', ' ') if len(df_header) > 0 and len(df_header.columns) > 0 else '',
    df_header.iloc[3, 1].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 1 else '',
    df_header.iloc[4, 2].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 2 else '',
    df_header.iloc[4, 3].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 3 else '',
    'Acum.'
]

df1.columns = novos_nomes + list(df1.columns[5:])
column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9, 10]
df1 = df1.iloc[:, column_order]

df2 = df.iloc[:, [0] + list(range(5, 10))].copy()
if len(df_header) > 0 and len(df_header.columns) > 5:
    df2['disease'] = df_header.iloc[0, 5].replace('\n', ' ')
    df2['year'] = df_header.iloc[1, 5].replace('\n', ' ')
else:
    df2['disease'] = ''
    df2['year'] = ''

if num_col == 12:
    
    df2['MF'] = df2.get('Coluna10', 0) + df2.get('Coluna11', 0)
    
else:
    
    df2['MF'] = df2.get('Coluna8', 0) + df2.get('Coluna9', 0)
    
    
df2['Acum_Year'] = int(df_header.iloc[1, 5].replace('\n', ' ')) - 1

df2['week'] = weeknumber

column_order = [0, 2, 3, 4, 8, 5, 9, 6, 7, 10, 1]

df2 = df2.iloc[:, column_order]

novos_nomes = [
    df_header.iloc[0, 0].replace('\n', ' ') if len(df_header) > 0 and len(df_header.columns) > 0 else '',
    df_header.iloc[3, 1].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 1 else '',
    df_header.iloc[4, 2].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 2 else '',
    df_header.iloc[4, 3].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 3 else '',
    'MF',
    'Acum.'
]

df2.columns = novos_nomes + list(df2.columns[6:10]) + ['En_Estudio']

df1

,ENTIDAD FEDERATIVA,Sem.,M,F,MF,Acum.,Acum_Year,disease,year,week,En_Estudio
0,Aguascalientes,4,1,8,9,0,2014,§Fiebre por Dengue CIE-10ª REV. A90,2015,43,
1,Baja California,34,33,55,88,2,2014,§Fiebre por Dengue CIE-10ª REV. A90,2015,43,
2,Baja California Sur,58,90,169,259,2979,2014,§Fiebre por Dengue CIE-10ª REV. A90,2015,43,
3,Campeche,0,146,197,343,366,2014,§Fiebre por Dengue CIE-10ª REV. A90,2015,43,
4,Coahuila,8,20,16,36,139,2014,§Fiebre por Dengue CIE-10ª REV. A90,2015,43,
5,Colima,5,430,611,1041,522,2014,§Fiebre por Dengue CIE-10ª REV. A90,2015,43,
6,Chiapas,8,403,531,934,694,2014,§Fiebre por Dengue CIE-10ª REV. A90,2015,43,
7,Chihuahua,5,3,7,10,0,2014,§Fiebre por Dengue CIE-10ª REV. A90,2015,43,
8,Distrito Federal,0,0,0,0,0,2014,§Fiebre por Dengue CIE-10ª REV. A90,2015,43,
9,Durango,5,45,74,119,35,2014,§Fiebre por Dengue CIE-10ª REV. A90,2015,43,


In [115]:
df_header

,0,1,2,3,4,5,6,7,8,9
0,ENTIDAD\nFEDERATIVA,§Fiebre por Dengue\nCIE-10ª REV.\nA90,None,None,None,§Fiebre Hemorrágica por Dengue\nCIE-10ª REV.\nA91,None,None,None,None
1,None,2015,None,None,2014,2015,None,None,None,2014
2,None,Confirmados,None,None,Confirmados\nAcum.,En Estudio\nAcum.,Confirmados,None,None,Confirmados\nAcum.
3,None,Sem.,Acum.,None,None,None,Sem.,Acum.,None,None
4,None,None,M,F,None,None,None,M,F,None
5,Aguascalientes 4\nBaja California 34\nBaja Cal...,None,1 8\n33 55\n90 169\n146 197\n20 16\n430 611\n4...,None,-\n2\n2 979\n366\n139\n522\n694\n-\n-\n35\n9\n...,-\n2\n4\n-\n3\n116\n-\n-\n-\n-\n-\n-\n-\n483\n...,-\n-\n-\n-\n-\n2\n12\n-\n-\n-\n-\n6\n-\n28\n-\...,1 -\n- -\n2 1\n41 35\n- -\n49 42\n161 177\n- 1...,None,-\n-\n77\n76\n5\n128\n737\n-\n-\n1\n-\n516\n-\...
6,TOTAL 835,None,6 283 9 440,None,15 818,2 612,197,1 843 2 049,None,5 951


In [18]:
not df_header.iloc[5, 0]

False

In [117]:
first = None

for n in range(4, 10):
    if dfx.iloc[n, 0]:
        first = n
        break
        
        
print(n)

5


In [92]:
len(df.columns)

12

In [116]:
dfx

,0,1,2,3,4,5,6,7,8,9
0,,,Confirmados,,Confirmados,En Estudio,,Confirmados,,Confirmados
1,,,Acum.,,,,,Acum.,,
2,,Sem.,,,Acum.,Acum.,Sem.,,,Acum.
3,,,,,,,,,F,
4,,,M,F,,,,M,,
5,Aguascalientes,4,1,8,-,-,-,1,-,-
6,Baja California,34,33,55,2,2,-,-,-,-
7,Baja California Sur,58,90,169,2 979,4,-,2,1,77
8,Campeche,-,146,197,366,-,-,41,35,76
9,Coahuila,8,20,16,139,3,-,-,-,5
